# Extract data from output files
### Analyze the output from a single LBANN run
March 9, 2020 \
April 6, 2020 : to store files in order of epochs \
April 21, 2020: added jupyter widgets to compare pixel intensity plots \
July 30, 2020: Perform analysis with stored histograms.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [ ]:
%matplotlib widget

In [ ]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [ ]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

## Extract folder

In [ ]:
img_size=128
dict1={'scratch':'/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size),
    'proj':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
display(u)


In [ ]:
parent_dir=u.result
dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'202*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

In [ ]:
result=w.result
main_dir=parent_dir+result
print(main_dir)

In [ ]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [ ]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1


### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,100.5,5),np.arange(100.5,300.5,20),np.arange(300.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

In [43]:
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/scaling_runs/20201031_205304_bsize512_scaling_2_8'

## Extract metrics info from log file

In [44]:
def f_extract_info(fname):
    '''
    Module to extract information from out.log files of Lbann training
    Reads in file name
    '''
    strg_lst=['objective','d_real','d_fake','gen','spec_loss','run time','mini-batch']
    keys=['training_'+strg for strg in strg_lst]
    dict1={}
    for category in ['training','validation']:
        for strg in strg_lst:
            try: 
                key=category+'_'+strg
                cmd='grep "{0}" {1} | grep "{2}"'.format(category,fname,strg)
        #         print(cmd)
                op1=sp.check_output(cmd,shell=True).decode('utf-8').split('\n')
                obj=np.array([strg.split(':')[-1] for strg in op1 if strg])
                dict1[key]=obj
            except Exception as e:
                print(e)
                dict1[key]=np.nan
    
    df=pd.DataFrame([])
    key_lst=['training_objective', 'training_d_real', 'training_d_fake', 'training_gen', 'training_spec_loss','validation_objective', 'validation_d_real', 'validation_d_fake', 'validation_gen','validation_spec_loss']
    col_list=['train_obj','train_dreal','train_dfake','train_gen','train_spec','val_obj','val_dreal','val_dfake','val_gen','val_spec']
    for col,key in zip(col_list,key_lst):
        try: 
            df[col]=dict1[key].astype(np.float)
        except: pass

    ### Need to remove the trailing 's' in the timings
    for col,key in zip(['train_time','val_time'],['training_run time','validation_run time']):
        df[col]=np.array([i[:-1] for i in dict1[key]]).astype(np.float)

    for col,key in zip(['train_batch_stats','val_batch_stats'],['training_mini-batch','validation_mini-batch']):
        df[col]=dict1[key]
        
    return df

def f_plot_metrics(df,col_list=['train_obj']):
    '''
    Plot multiple columns of the dataframe
    '''
    plt.figure()
    
    marker_lst=('o','*','H','D','.','x')
    marker=itertools.cycle(marker_lst)
    for col in col_list: plt.plot(df[col],linestyle='',marker=next(marker),label=col)
    plt.legend()
    plt.xlabel('Epoch')
    

In [45]:
strg=main_dir+'/out.log'
df_metrics=f_extract_info(glob.glob(strg)[0])

In [46]:
df_metrics.head()

,train_obj,train_dreal,train_dfake,train_gen,train_spec,val_obj,val_dreal,val_dfake,val_gen,val_spec,train_time,val_time,train_batch_stats,val_batch_stats
0,13.56810,1.264900,1.154540,9.80116,13.4753,2.19934,0.281325,0.536527,0.014399,13.6709,173.761,3.74762,"0.197449s mean, 3.94145s max, 0.0302313s min,...","0.0381801s mean, 1.15027s max, 0.0198753s min..."
1,6.74166,0.473388,0.424663,4.62894,12.1467,28.30710,0.101750,0.130012,26.707600,13.6776,167.910,2.22450,"0.190798s mean, 3.77667s max, 0.0256201s min,...","0.0226465s mean, 0.0495297s max, 0.0148799s m..."
2,6.51110,0.349436,0.322044,4.64908,11.9054,6.90543,0.098770,3.802310,1.767640,12.3670,174.106,2.20900,"0.197851s mean, 4.13544s max, 0.024362s min, ...","0.0224729s mean, 0.0508726s max, 0.0149095s m..."
3,6.22077,0.320762,0.301277,4.42571,11.7302,7.31053,0.086668,0.130797,5.900600,11.9246,166.671,2.23297,"0.189392s mean, 3.21994s max, 0.0244294s min,...","0.0227339s mean, 0.0507553s max, 0.015235s mi..."
4,5.71714,0.332931,0.324159,3.88643,11.7362,21.66120,4.101680,0.121550,16.256800,11.8114,167.893,2.39839,"0.190784s mean, 3.97556s max, 0.0252403s min,...","0.0244219s mean, 0.265225s max, 0.0148283s mi..."


In [47]:
col_list=df_metrics.columns[~df_metrics.columns.str.endswith('stats')]
interact_manual(f_plot_metrics,col_list=SelectMultiple(options=col_list),df=fixed(df_metrics))


interactive(children=(SelectMultiple(description='col_list', options=('train_obj', 'train_dreal', 'train_dfake…

<function __main__.f_plot_metrics(df, col_list=['train_obj'])>

## Extract data from stored dataframe

In [48]:
### Load data
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [49]:
print(df.shape)
df.tail()


(3516, 22)


,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,...,chi_spec3,num_imgs,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_sdev,label
3511,39,35110,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.035826,0.032750,0.243801,0.312377,0.007125,5.807954,...,26.365186,256,19,7,"[1.0487138163163268, 0.4780254855185938, 0.045...","[0.0037032377406290957, 0.001730421208449659, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[87462.74302036758, 61455.0704494546, 45497.91...","[45105.62380118491, 25693.168101121515, 17665....",39-35110
3512,39,35120,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.108443,0.896366,20.245580,21.250389,0.015212,20.190483,...,39.687845,256,85,23,"[1.1868982554437415, 0.38960823326085925, 0.05...","[0.00640672608555894, 0.002723326310628461, 0....","[-1.031746031584782, -0.3099415213241682, 0.30...","[243543.95415830612, 139485.95173084136, 79434...","[181589.9479330393, 76278.96898891008, 41815.9...",39-35120
3513,39,35130,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.007277,0.007883,0.036876,0.052036,0.003559,5.835237,...,29.681164,256,4,0,"[1.0656812069485715, 0.4632568050040181, 0.053...","[0.003833220931113306, 0.001804598979261642, 0...","[-1.031746031584782, -0.3099415213241682, 0.30...","[105417.6754655242, 72423.40978275459, 49019.8...","[52754.97139532524, 24637.981920873106, 15290....",39-35130
3514,39,35140,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.029623,0.019422,0.010155,0.059201,0.014448,4.961370,...,29.296608,256,1,0,"[1.0146527838056478, 0.49599273690270484, 0.04...","[0.0037808503805523168, 0.0017975625864859776,...","[-1.031746031584782, -0.3099415213241682, 0.30...","[107545.50756015256, 64100.41166384026, 38201....","[46923.88254189833, 22613.14596657754, 12000.1...",39-35140
3515,39,35150,train_gen,/global/cscratch1/sd/vpa/proj/cosmogan/results...,0.210944,0.588440,3.746355,4.545738,0.023435,14.956978,...,40.435208,256,42,4,"[1.224590344795817, 0.3646504568993883, 0.0633...","[0.006168095610293088, 0.0025929487558583658, ...","[-1.031746031584782, -0.3099415213241682, 0.30...","[317101.0415375903, 150070.7877906014, 77988.3...","[193844.95616011403, 65097.08753183364, 30762....",39-35150


In [50]:
df.columns

Index(['epoch', 'step', 'img_type', 'fname', 'chi_1a', 'chi_1b', 'chi_1c',
       'chi_1', 'chi_2', 'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3',
       'num_imgs', 'num_large', 'num_vlarge', 'hist_val', 'hist_err',
       'hist_bin_centers', 'spec_val', 'spec_sdev', 'label'],
      dtype='object')

### View best epochs

#### Locations with best chi_sqr

In [51]:
def f_get_best_chisqr_models(df,cutoff=0.2):
    '''
    Pick models with lowest 20% chi-square for multiple categories
    '''
    chi_sqr_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[chi_sqr_keys])
#     print(q_dict)
    
    df_sliced=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    return df_sliced

chi_sqr_keys=['epoch','step','chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']
# index location of min/max values of chi squares
inds=[]
for key in ['chi_1a','chi_1b','chi_1c','chi_1','chi_2','chi_imgvar','chi_spec1','chi_spec2']:
    inds.append(df[key].idxmin(axis=1))
inds=np.array(inds)
df.loc[inds][chi_sqr_keys]


,epoch,step,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2
1311,14,13110,0.000286,0.001523,0.028037,0.029846,0.000021,5.844486,0.964253,92.027591
2428,27,24280,0.031355,0.000465,0.080078,0.111898,0.023552,6.835513,1.450423,168.385562
2151,24,21510,0.017324,0.011970,0.000362,0.029655,0.012182,4.937638,1.535092,147.697091
1425,16,14250,0.000944,0.005405,0.003801,0.010150,0.000087,5.198988,1.015935,129.752758
1311,14,13110,0.000286,0.001523,0.028037,0.029846,0.000021,5.844486,0.964253,92.027591
35,0,350,0.731436,0.108666,0.003864,0.843966,0.476141,0.835156,1019.573822,38937.629665
1318,14,13180,0.005926,0.021521,0.001881,0.029328,0.000451,4.357457,0.209072,21.188544
1592,18,15920,0.041225,0.022784,0.002258,0.066266,0.028728,4.731339,0.227437,17.244252


In [52]:
### Plot chi-sqr values
# _=df_sliced.plot(x="step", y=["chi_1", "chi_imgvar", "chi_spec1"],style='.',marker='*')

In [53]:
### Plot number of very high pixel images
plt.figure()
plt.plot(df[df.img_type=='train_gen'].step,df[df.img_type=='train_gen'].num_vlarge,linestyle='',marker='*')
plt.xlabel('Steps in Epochs')
plt.ylabel('Number of very large pixel images from a batch of images')

# df[df.num_vlarge>10].tail()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Number of very large pixel images from a batch of images')

### View best steps

In [54]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [55]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [57]:
df_sliced=w.result
# df_sliced

### Pick best steps

In [58]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1b',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.2,sort_col='chi_spec1',head=4,display_flag=False).step.values)

best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.048586440664743796, 'chi_spec1': 0.9595820821937462}
[0, 39]
{'chi_1': 0.18205381196932835, 'chi_spec1': 4.161669818839055}
[0, 39]
{'chi_1': 0.048586440664743796, 'chi_spec1': 0.9595820821937462}
[ 8370  9340  9740 12490 12900 13180 13200 14130 16320 24280]


In [59]:
# best_step=[3430,  6460,  7215,  9610, 11725, 14635, 9475]
# best_step=np.arange(40130,40135).astype(int)

In [60]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(10, 22)
[(9, 8370), (10, 9340), (11, 9740), (14, 12490), (14, 12900), (14, 13180), (15, 13200), (16, 14130), (18, 16320), (27, 24280)]


In [61]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
837,9-8370,0.012404,0.572098,28.119365,0.003101,0.001572,0.007731,0.000389,5.845777,65.281134,9,8370
934,10-9340,0.028776,1.016706,32.213418,0.016826,0.000544,0.011406,0.005602,6.155468,79.214430,10,9340
974,11-9740,0.012856,0.439721,24.589320,0.003304,0.006190,0.003362,0.001042,5.367010,51.218446,11,9740
1249,14-12490,0.024431,0.274913,29.465780,0.002288,0.011293,0.010850,0.000132,5.416338,28.610146,14,12490
1290,14-12900,0.028637,0.243950,29.714326,0.001633,0.005733,0.021271,0.000105,6.097034,23.293004,14,12900
1318,14-13180,0.029328,0.209072,27.125000,0.005926,0.021521,0.001881,0.000451,4.357457,21.188544,14,13180
1320,15-13200,0.010161,0.927037,25.078629,0.000915,0.000955,0.008291,0.000030,5.742977,120.562803,15,13200
1413,16-14130,0.041902,0.256102,26.813182,0.005148,0.002324,0.034430,0.003570,6.006676,21.003813,16,14130
1632,18-16320,0.014056,0.714743,22.537641,0.004162,0.007444,0.002451,0.002871,5.201799,86.015834,18,16320
2428,27-24280,0.111898,1.450423,25.632244,0.031355,0.000465,0.080078,0.023552,6.835513,168.385562,27,24280


## Plot pixel intensity and spectrum

In [62]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))

    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            
        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,0,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=100)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')

    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [63]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','spec_relative','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('9-8370', '10-9340', '11-9740', '14-1…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

In [31]:
df[(df.step>8000)& (df.step<10000)].plot(kind='line',x='step',y=['chi_1','chi_spec1'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='step'>

## View image block

In [32]:
def f_get_img(df,step):
    df_temp=df[(df.step==step)]
    images=np.load(df_temp.fname.values[0])[:,0,:,:]
    return images

img_arr=f_get_img(df,8020)
# f_plot_grid(img_arr[:18],cols=6,fig_size=(10,5))

In [33]:
f_plot_grid(s_val[100:118],cols=6,fig_size=(10,5))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting spread of spectrum of input data

In [34]:

# plt.figure()

# idxs=np.random.randint(0,s_val.shape[0],size=50)
# arr=s_val[idxs]
# Pk = f_batch_spectrum(arr)
# k=np.arange(Pk.shape[1])
# for count,i in enumerate(Pk):
#     plt.plot(i)
    
# Pk = f_batch_spectrum(img_arr)
# mean,std = np.mean(Pk, axis=0),np.std(Pk, axis=0)/np.sqrt(Pk.shape[0])
# plt.plot(k, mean, 'k:',label='generated image')


# plt.legend()

# plt.ylabel(r'$P(k)$')
# plt.xlabel(r'$k$')
# plt.title('Power Spectrum: 50 input images')
# plt.yscale('log')



In [35]:
main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/'
f1=main_dir+'20201208_165658_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a1=np.load(f1)[:,0,:,:]
f2=main_dir+'20201209_055955_bsize256_scale0.1_deterministic_on/dump_outs/trainer0/model0/sgd.training.epoch.1.step.1000_gen_img_instance1_activation_output0.npy'
a2=np.load(f2)[:,0,:,:]

print(a1.shape)
print(a2.shape)


(256, 128, 128)
(256, 128, 128)


In [36]:
# f_pixel_intensity(a1)
# f_pixel_intensity(a2)

f_compare_pixel_intensity([a1,a2],label_lst=['img1','img2'],bkgnd_arr=[],log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:153: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important
/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:196: MatplotlibDeprecationWarning: The 'linthreshx' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  plt.xscale('symlog',linthreshx=50)


In [37]:
df_best.columns

Index(['epoch', 'step', 'img_type', 'fname', 'chi_1a', 'chi_1b', 'chi_1c',
       'chi_1', 'chi_2', 'chi_imgvar', 'chi_spec1', 'chi_spec2', 'chi_spec3',
       'num_imgs', 'num_large', 'num_vlarge', 'hist_val', 'hist_err',
       'hist_bin_centers', 'spec_val', 'spec_sdev', 'label'],
      dtype='object')

In [38]:
f_invtransform(df_best.iloc[0].hist_bin_centers)


array([-6.24999994e-02,  2.10714284e+00,  7.47916654e+00,  1.26323526e+01,
        1.77159086e+01,  2.27685177e+01,  2.78046863e+01,  3.28310795e+01,
        3.78511884e+01,  4.28670187e+01,  4.78798045e+01,  5.28903470e+01,
        5.78991889e+01,  6.29067110e+01,  6.79131882e+01,  7.29188240e+01,
        7.79237723e+01,  8.29281517e+01,  8.79320549e+01,  9.29355555e+01,
        9.79387128e+01,  1.09626622e+02,  1.29756484e+02,  1.49852722e+02,
        1.69926897e+02,  1.89985815e+02,  2.10033743e+02,  2.30073493e+02,
        2.50106993e+02,  2.70135608e+02,  2.90160334e+02,  3.23603053e+02,
        3.73852919e+02,  4.24044592e+02,  4.74196274e+02,  5.24319294e+02,
        5.74421067e+02,  6.24506656e+02,  6.74579632e+02,  7.24642587e+02,
        7.74697448e+02,  8.24745678e+02,  8.74788406e+02,  9.24826519e+02,
        1.28909779e+03])